In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
with open("preprocessed_data.pkl", "rb") as f:
    data = pickle.load(f)

X_train = data["X_train"].values
X_val   = data["X_val"].values
X_test  = data["X_test"].values
y_train = data["y_train"]
y_val   = data["y_val"]
y_test  = data["y_test"]

In [ ]:
def create_sequences(X, y, seq_len=30):
    X_seq = []
    y_seq = []

    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i+seq_len])

    return np.array(X_seq), np.array(y_seq)



In [ ]:
SEQ_LEN = 30  # window size

X_train_seq, y_train_seq = create_sequences(X_train, y_train, SEQ_LEN)
X_val_seq,   y_val_seq   = create_sequences(X_val, y_val, SEQ_LEN)
X_test_seq,  y_test_seq  = create_sequences(X_test, y_test, SEQ_LEN)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[SEQ_LEN, X_train_seq.shape[2]]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(
        filters=32,
        kernel_size=2,
        padding="causal",
        activation="relu",
        dilation_rate=rate
    ))

model.add(keras.layers.Conv1D(filters=1, kernel_size=1))

# Flatten for regression output
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1))


model.compile(loss="mse", optimizer="adam", metrics=["mae"])


In [ ]:
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=20,
    batch_size=64
)


In [ ]:
preds = model.predict(X_test_seq).flatten()
rmse = np.sqrt(mean_squared_error(y_test_seq, preds))

print("WaveNet RMSE:", rmse)


In [ ]:
plt.figure(figsize=(12,5))
plt.plot(y_test_seq[:200], label="True")
plt.plot(preds[:200], label="Predicted")
plt.title("WaveNet Predictions")
plt.legend()
plt.show()